In [3]:
cd drive/MyDrive/OwlDetection/

[Errno 2] No such file or directory: 'drive/MyDrive/OwlDetection/'
/content/drive/MyDrive/OwlDetection


In [5]:
!git config --global user.email "sabedi4@uwo.ca"
!git config --global user.name "saeidabedi76"


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

# Set some parameters
img_height, img_width = 224, 224
batch_size = 32

# Set up the training, validation, and test directories
train_dir = '/content/drive/MyDrive/OwlDetection/train'
validation_dir = '/content/drive/MyDrive/OwlDetection/valid'
test_dir = '/content/drive/MyDrive/OwlDetection/test'

# Set up data augmentation configuration for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Note that validation and test data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1./255)

# Set up the data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'  # 'categorical' because we have more than 2 classes
)

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # Do not shuffle test data
)

# Helper function to plot images
def plot_images(images_arr, titles):
    fig, axes = plt.subplots(1, len(images_arr), figsize=(20,20))
    axes = axes.flatten()
    for img, ax, title in zip(images_arr, axes, titles):
        ax.imshow(img)
        ax.axis('off')
        ax.set_title(title)
    plt.tight_layout()
    plt.show()

# Displaying images from each generator
# Extract and plot a batch from each
train_images, _ = next(train_generator)
validation_images, _ = next(validation_generator)
test_images, _ = next(test_generator)

# We take the first three images of the batch
plot_images(train_images[:3], ['Train1', 'Train2', 'Train3'])
plot_images(validation_images[:3], ['Validation1', 'Validation2', 'Validation3'])
plot_images(test_images[:3], ['Test1', 'Test2', 'Test3'])


Found 923 images belonging to 6 classes.
Found 30 images belonging to 6 classes.
Found 30 images belonging to 6 classes.


CLIP Zero-shot as the standard of the other model's accuracy

In [ ]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch
import os

# Load CLIP model and processor from Hugging Face Transformers
model_id = "openai/clip-vit-base-patch32"
model = CLIPModel.from_pretrained(model_id)
processor = CLIPProcessor.from_pretrained(model_id)

# Directory containing your dataset
dataset_path = train_dir  # Update this path if needed

# Class labels
class_folders = ['BARN OWL', 'GREAT GRAY OWL', 'LONG-EARED OWL', 'ORIENTAL BAY OWL', 'SNOWY OWL', 'STRIPED OWL']

# Initialize counters
correct_predictions = 0
total_images = 0

# Loop through each class folder
for class_name in class_folders:
    folder_path = os.path.join(dataset_path, class_name)

    # Check if the folder exists
    if not os.path.isdir(folder_path):
        print(f"Folder {folder_path} does not exist.")
        continue

    for image_name in os.listdir(folder_path):
        # Prepare the image and text inputs
        image_path = os.path.join(folder_path, image_name)
        try:
            image = Image.open(image_path).convert("RGB")
        except (IOError, FileNotFoundError):
            print(f"Error opening image {image_path}. Skipping.")
            continue

        inputs = processor(text=class_folders, images=image, return_tensors="pt", padding=True)

        # Get model outputs
        with torch.no_grad():
            outputs = model(**inputs)

        # Process outputs (calculate probabilities and find the best match)
        logits_per_image = outputs.logits_per_image
        probs = logits_per_image.softmax(dim=1)
        top_result = probs.argmax()

        # Check prediction
        predicted_class = class_folders[top_result]
        if predicted_class == class_name:
            correct_predictions += 1
        total_images += 1

        print(f"Image: {image_name}, Actual class: {class_name}, Predicted: {predicted_class}")

# Calculate accuracy
accuracy = correct_predictions / total_images if total_images > 0 else 0
print(f"Overall accuracy: {accuracy * 100:.2f}%")